In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.nn import LeakyReLU
from torch.utils.tensorboard import SummaryWriter

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 10

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_val_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

valset , testset = torch.utils.data.random_split(test_val_set, [0.4, 0.6])

valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

44.5%


KeyboardInterrupt: 

In [14]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
#        self.soft = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(nn.LeakyReLU()(self.conv1(x)))
        x = self.pool(nn.LeakyReLU()(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.LeakyReLU()(self.fc1(x))
        x = nn.LeakyReLU()(self.fc2(x))
        x = self.fc3(x)
#        x = self.soft
        return x
    
    
net = Net()

In [11]:
def train_model(train_losses, train_accuracy, validation_losses, validation_accuracy, epochs):
    best_loss = 100
    for epoch in range(epochs):
        correct = 0
        total = 0
        accuracy = 0
        for i, data in enumerate(trainloader, 0):

            inputs, labels = data

            outputs = net(inputs) #prediction

            if (outputs == inputs):
                correct +=1
        
            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            total += 1

        train_losses.append(loss.item())
        accuracy = correct/total
        train_accuracy.append(accuracy.item())

    


    return train_losses, train_accuracy, validation_losses, validation_accuracy

In [15]:

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.0001)

train_losses = []
validation_losses = []
train_accuracy = []
validation_accuracy = []
epochs = 30


train_losses, train_accuracy, validation_losses, validation_accuracy = train_model(train_losses, train_accuracy, validation_losses, validation_accuracy, epochs)

writer = SummaryWriter()

writer.add_scalar('Loss/train', train_losses(), range(epochs))
writer.add_scalar('Accuracy/train', train_accuracy(), range(epochs))
